In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename='Boston Neighborhood Coordinates.xlsx', read_only=True)
ws = wb.active
col_names = [ws["A1"].value, ws["B1"].value, ws["C1"].value, ws["D1"].value]
header = np.array(col_names)

df = pd.DataFrame(ws.values, columns=header)
wb.close()

In [3]:
df = df.drop(['Zip Code'], axis=1)
df = df.drop([0])
df.set_index('Neighborhood', inplace=True)

In [4]:
df = df.dropna()

In [5]:
df

,Latitude,Longitude
Neighborhood,,
Chinatown/Leather District,42.3515,-71.0602
West End,42.3653,-71.0646
Bay Village,42.3531,-71.0765
South End,42.3378,-71.0706
Roxbury,42.3218,-71.0884
Mission Hill,42.3319,-71.0988
Mattapan,42.2771,-71.0914
South Boston,42.3365,-71.0349
East Boston,42.3695,-71.0201


In [6]:
import folium # map rendering library

In [7]:
# create map of New York using latitude and longitude values
map_Boston = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)  
    
map_Boston

In [8]:
import requests
import json # library to handle JSON files

In [9]:
CLIENT_ID = 'GKVJ5KSFU1RD00P24CV5JT2XSARMWSONFYRMLT2EK2IJFEBW' # your Foursquare ID
CLIENT_SECRET = 'SSJALJ4S5QXDNQBZMNUNFBE1XMVQZETMJWU1JDXE2VWK2PZL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [10]:
hood_name = 'West End'
hood_lat = df.loc[hood_name, 'Latitude'] # neighborhood latitude value
hood_long = df.loc[hood_name, 'Longitude'] # neighborhood longitude value
radius = 500
LIMIT = 100

url = f'https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={hood_lat},{hood_long}&v={VERSION}&radius={radius}&limit={LIMIT}'

In [11]:
r = requests.get(url)
r.status_code
results = r.json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TD Garden,Hockey Arena,42.366329,-71.062153
1,Bobby Orr Statue @ TD Garden,Outdoor Sculpture,42.365314,-71.062381
2,Tasty Burger,Burger Joint,42.365738,-71.063105
3,The Clubs at Charles River Park,Gym / Fitness Center,42.365985,-71.068257
4,J. Pace & Son,Deli / Bodega,42.364682,-71.067107


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Boston_venues = getNearbyVenues(names=df.index, latitudes=df['Latitude'], longitudes=df['Longitude'])

Chinatown/Leather District
West End
Bay Village
South End
Roxbury
Mission Hill
Mattapan
South Boston
East Boston
Charlestown
Jamaica Plain
Roslindale
West Roxbury
Allston
Brighton
Hyde Park
North Cambridge 
Kendall Square
Brookline Village
Chestnut Hill
Downtown
North End
Beacon Hill
Back Bay
Dorchester
Harvard Square 
East Cambridge
Fenway/Kenmore
Porter Square
West Cambridge
Inman Square
Central Square
Davis Square
Winter Hill
Assembly Square
Seaport


In [17]:
print(Boston_venues.shape)
    
Boston_venues.nunique()

(1421, 7)


Neighborhood                36
Neighborhood Latitude       36
Neighborhood Longitude      32
Venue                     1190
Venue Latitude            1343
Venue Longitude           1344
Venue Category             260
dtype: int64

In [18]:
Boston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown/Leather District,42.3515,-71.0602,Gracenote Coffee,42.351657,-71.058204,Coffee Shop
1,Chinatown/Leather District,42.3515,-71.0602,Shabu Zen,42.350922,-71.060942,Asian Restaurant
2,Chinatown/Leather District,42.3515,-71.0602,Crave - Mad for Chicken,42.350378,-71.060786,Korean Restaurant
3,Chinatown/Leather District,42.3515,-71.0602,Shojo,42.351028,-71.060736,Asian Restaurant
4,Chinatown/Leather District,42.3515,-71.0602,Hostelling International Boston,42.351084,-71.063398,Hostel


In [19]:
Venue_cat_count = Boston_venues.groupby('Venue Category').count()
Venue_cat_count.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_cat_count = pd.DataFrame(Venue_cat_count['Count'], Venue_cat_count.index)
Venue_cat_count

,Count
Venue Category,
ATM,1
Accessories Store,3
Afghan Restaurant,2
African Restaurant,1
Airport,1
...,...
Wine Bar,9
Wine Shop,3
Wings Joint,1


In [20]:
Venue_cat_count.to_excel('Venue_Category_Counts.xlsx')

In [21]:
Venue_per_hood = Boston_venues.groupby('Neighborhood').count()
Venue_per_hood.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_per_hood = pd.DataFrame(Venue_per_hood['Count'], Venue_per_hood.index)
Venue_per_hood

,Count
Neighborhood,
Allston,8
Assembly Square,85
Back Bay,56
Bay Village,51
Beacon Hill,41
Brighton,15
Brookline Village,4
Central Square,39
Charlestown,25


In [22]:
# one hot encoding
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Boston_onehot['Neighborhood'] = Boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

Totals = Boston_onehot.sum()
Totals = pd.DataFrame(Totals)
Totals = Totals.drop('Neighborhood')
Totals.rename(columns={0: 'Totals'}, inplace=True)

In [23]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allston,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Assembly Square,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529,...,0.000000,0.00000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529
2,Back Bay,0.017857,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bay Village,0.000000,0.00000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,...,0.019608,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Beacon Hill,0.024390,0.02439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Brighton,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000
6,Brookline Village,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Square,0.025641,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,...,0.076923,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Charlestown,0.080000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Chestnut Hill,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
num_top_venues = 10

for hood in Boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Boston_grouped[Boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                           venue  freq
0                     Donut Shop  0.25
1            Rental Car Location  0.25
2                         Bakery  0.12
3                            Gym  0.12
4                   Liquor Store  0.12
5                           Café  0.12
6                    Yoga Studio  0.00
7              Outdoor Sculpture  0.00
8  Paper / Office Supplies Store  0.00
9                   Outlet Store  0.00


----Assembly Square----
                    venue  freq
0          Clothing Store  0.11
1             Coffee Shop  0.04
2     Sporting Goods Shop  0.04
3           Women's Store  0.02
4              Shoe Store  0.02
5                    Café  0.02
6        Department Store  0.02
7  Furniture / Home Store  0.02
8                     Gym  0.02
9                    Park  0.02


----Back Bay----
                      venue  freq
0        Italian Restaurant  0.05
1                     Hotel  0.05
2        Seafood Restaurant  0.05
3               Coffee

                venue  freq
0          Donut Shop  0.07
1               Hotel  0.07
2      Sandwich Place  0.07
3                 Bar  0.05
4  Mexican Restaurant  0.04
5          Sports Bar  0.04
6         Coffee Shop  0.04
7         Pizza Place  0.04
8                Café  0.03
9   Convenience Store  0.03


----West Roxbury----
               venue  freq
0      Grocery Store  0.17
1                Pub  0.08
2           Pharmacy  0.08
3  Convenience Store  0.08
4        Gas Station  0.08
5    Automotive Shop  0.08
6         Donut Shop  0.08
7        Pizza Place  0.08
8               Bank  0.08
9       Liquor Store  0.08


----Winter Hill----
                   venue  freq
0            Pizza Place  0.16
1           Liquor Store  0.11
2        Meze Restaurant  0.05
3          Grocery Store  0.05
4         Sandwich Place  0.05
5               Pharmacy  0.05
6                   Café  0.05
7         Ice Cream Shop  0.05
8  General Entertainment  0.05
9     Falafel Restaurant  0.05




In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Rental Car Location,Donut Shop,Café,Liquor Store,Bakery,Gym,Farmers Market,Fast Food Restaurant,Falafel Restaurant,Dog Run
1,Assembly Square,Clothing Store,Sporting Goods Shop,Coffee Shop,Women's Store,Café,Gym,Furniture / Home Store,Outlet Store,Department Store,Park
2,Back Bay,Hotel,Italian Restaurant,Seafood Restaurant,Coffee Shop,Clothing Store,Plaza,Ice Cream Shop,Grocery Store,Mediterranean Restaurant,Restaurant
3,Bay Village,Jewelry Store,Spa,Park,Hotel,Plaza,Salad Place,Clothing Store,Coffee Shop,Farmers Market,Bagel Shop
4,Beacon Hill,Pizza Place,Italian Restaurant,French Restaurant,Gourmet Shop,Food Truck,Other Repair Shop,Breakfast Spot,Burrito Place,Café,Restaurant
5,Brighton,Pizza Place,Supplement Shop,Gift Shop,Wings Joint,Mobile Phone Shop,Coffee Shop,Convenience Store,Salon / Barbershop,Grocery Store,Lake
6,Brookline Village,Lake,Platform,Park,Beer Garden,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
7,Central Square,Vegetarian / Vegan Restaurant,Sandwich Place,Bar,Cocktail Bar,Nightclub,Japanese Restaurant,Hotpot Restaurant,Shanghai Restaurant,Rock Club,Racetrack
8,Charlestown,Yoga Studio,Pizza Place,Coffee Shop,Athletics & Sports,Shopping Mall,National Park,Donut Shop,Café,Gastropub,Bank
9,Chestnut Hill,Park,Playground,Women's Store,Exhibit,Donut Shop,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [117]:
def df_style(val):
    boolean = 0
    for item in match:
        if val == item:
            boolean = 1
        
    if boolean == 1:
        return 'font-weight: bold'
    else:
        return ''

In [196]:
criterion = ['Sports', 'Pizza', 'Donut']
match = []
search_results = neighborhoods_venues_sorted

for item in criterion:

    mask = search_results.applymap(lambda x: item in str(x))
    search_results = search_results[mask == True].dropna(how='all')
    
    for i in range(0,len(search_results.index)):
        results = []
        results = search_results.iloc[i,].unique()
        match.insert(0, results[1])
    match = list(dict.fromkeys(match))
    
    search_results = neighborhoods_venues_sorted.iloc[search_results.index]
    
i=0
for items in zip(search_results['Neighborhood']):
    i=i+1
print(f"{i} Results Found!")

search_results.style.applymap(df_style)

2 Results Found!


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Number of Matches
8,Charlestown,Yoga Studio,Pizza Place,Coffee Shop,Athletics & Sports,Shopping Mall,National Park,Donut Shop,Café,Gastropub,Bank,3
33,West End,Donut Shop,Sandwich Place,Hotel,Bar,Coffee Shop,Pizza Place,Mexican Restaurant,Sports Bar,Convenience Store,Café,3


In [195]:
num_match = []
for row in neighborhoods_venues_sorted.itertuples():
    i=0
    for item in zip(row):
        for crit in match:
            if item[0] == crit:
                i=i+1
    num_match.append(i)
neighborhoods_venues_sorted['Number of Matches'] = num_match

neighborhoods_venues_sorted.style.applymap(df_style)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Number of Matches
0,Allston,Rental Car Location,Donut Shop,Café,Liquor Store,Bakery,Gym,Farmers Market,Fast Food Restaurant,Falafel Restaurant,Dog Run,1
1,Assembly Square,Clothing Store,Sporting Goods Shop,Coffee Shop,Women's Store,Café,Gym,Furniture / Home Store,Outlet Store,Department Store,Park,0
2,Back Bay,Hotel,Italian Restaurant,Seafood Restaurant,Coffee Shop,Clothing Store,Plaza,Ice Cream Shop,Grocery Store,Mediterranean Restaurant,Restaurant,0
3,Bay Village,Jewelry Store,Spa,Park,Hotel,Plaza,Salad Place,Clothing Store,Coffee Shop,Farmers Market,Bagel Shop,0
4,Beacon Hill,Pizza Place,Italian Restaurant,French Restaurant,Gourmet Shop,Food Truck,Other Repair Shop,Breakfast Spot,Burrito Place,Café,Restaurant,1
5,Brighton,Pizza Place,Supplement Shop,Gift Shop,Wings Joint,Mobile Phone Shop,Coffee Shop,Convenience Store,Salon / Barbershop,Grocery Store,Lake,1
6,Brookline Village,Lake,Platform,Park,Beer Garden,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,0
7,Central Square,Vegetarian / Vegan Restaurant,Sandwich Place,Bar,Cocktail Bar,Nightclub,Japanese Restaurant,Hotpot Restaurant,Shanghai Restaurant,Rock Club,Racetrack,0
8,Charlestown,Yoga Studio,Pizza Place,Coffee Shop,Athletics & Sports,Shopping Mall,National Park,Donut Shop,Café,Gastropub,Bank,3
9,Chestnut Hill,Park,Playground,Women's Store,Exhibit,Donut Shop,Duty-free Shop,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,1
